In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1', creation_time=1732838050542, experiment_id='1', last_update_time=1732838050542, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [6]:
client.create_experiment(name="my-cool-experiment")

'2'

In [18]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 5.4',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [19]:
for run in runs:
    print(f'run id: {run.info.run_id},  rmse: {run.data.metrics["rmse"]:.4f}')

run id: 7b3b188d2bec4c5b98f00e22a647c2ce,  rmse: 5.2795
run id: a3775d56b3c24c459915d8a9033da11e,  rmse: 5.3464
run id: 40eaa55556014c19982f2bce4ee97fa6,  rmse: 5.3467
run id: edd53062e7534c10aa6d34341e6261ae,  rmse: 5.3494


In [23]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [34]:
run_id = '7b3b188d2bec4c5b98f00e22a647c2ce'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [27]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_25391/1481491681.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1733120110651, current_stage='None', description=None, last_updated_timestamp=1733120110651, name='nyc-taxi-regressor', run_id='edd53062e7534c10aa6d34341e6261ae', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1/edd53062e7534c10aa6d34341e6261ae/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [31]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [32]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')

In [35]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment_tracking/preprocessor'

In [36]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [37]:
X_test = preprocess(df, dv)

In [38]:
target = "duration"
y_test = df[target].values